In [ ]:
import pandas as pd
import numpy as np
ruta_input = "../file/output/"
df_Estudiante = pd.read_csv(ruta_input+'SQL_tabla_1.csv',index_col=False, skiprows=2)
df_Ciudad = pd.read_csv(ruta_input+'SQL_tabla_2.csv', skiprows=2)
ruta_output_clean = "../data/"

In [ ]:
df_Estudiante.info()

In [ ]:
if df_Estudiante['ID'].is_unique:
    print("El ID de estudiante es único.")

carreras = df_Estudiante['CARRERA'].unique()
print("Carreras únicas:", carreras)


In [ ]:
dominios_default = df_Estudiante['CORREO'].str.split('@').str[1]
print("Dominios por defecto:", dominios_default.unique())

casos_sin_arroba = df_Estudiante[~df_Estudiante['CORREO'].str.contains('@', na=False)][['ID', 'CORREO']]
print(casos_sin_arroba)

# Remplazara correo de "ID == 6" por "odyuliana_@hotmail.com"
df_Estudiante.loc[df_Estudiante['ID'] == 6, 'CORREO'] = 'odyuliana_@hotmail.com'
df_Estudiante.loc[df_Estudiante['ID'] == 6, 'CONTACTO_INCOMPLETO'] = 'CORREO'

In [ ]:
# Estos dominios hotmail, gmail, outlook son validos sin terminan en .com 
df_Estudiante['CORREO'] = df_Estudiante['CORREO'].str.replace(r'@.*(hotmail|gmail|outlook).*', r'@\1.com', regex=True).where(df_Estudiante['CORREO'].str.contains(r'@(hotmail|gmail|outlook)', na=False))

In [ ]:
correos_a_reemplazar = df_Estudiante[df_Estudiante['CORREO'].isin(['@gmail.com', '@hotmail.com', '@outlook.com'])][['ID', 'CORREO']]
print("Correos a reemplazar:\n", correos_a_reemplazar)
df_Estudiante.loc[df_Estudiante['ID'].isin(correos_a_reemplazar['ID']), 'CORREO'] = np.nan

In [ ]:
df = df_Estudiante['TELEFONO'].astype(str).str.replace(r'\D', '', regex=True).str.len()

conteo_digitos = df.value_counts().sort_index()
print("Cantidad de dígitos y frecuencia:")
for digitos, cantidad in conteo_digitos.items():
    print(f"{digitos} dígitos: {cantidad} teléfonos")

print(f"\nMás común: {conteo_digitos.index[conteo_digitos.argmax()]} dígitos")


In [ ]:
# formatear la columana df_Estudiante['TELEFONO'] a entero si tiene 7 digitos, sino convertirlo a NaN
df_Estudiante['TELEFONO'] = df_Estudiante['TELEFONO'].astype(str).str.replace(r'\D', '', regex=True)
df_Estudiante['TELEFONO'] = df_Estudiante['TELEFONO'].apply(lambda x: int(x) if len(x) == 7 else pd.NA)
df_Estudiante.info()

In [ ]:
# FECHAINGRESO: object → debería ser datetime sin tener en cuenta la hora
df_Estudiante['FECHAINGRESO'] = pd.to_datetime(df_Estudiante['FECHAINGRESO'], errors='coerce').dt.date
condiciones = [
    df_Estudiante['FECHAINGRESO'].isnull(),
    df_Estudiante['TELEFONO'].isnull(),
    df_Estudiante['CORREO'].isnull()
]

opciones = ['FECHAINGRESO', 'TELEFONO', 'CORREO']
df_Estudiante['CONTACTO_INCOMPLETO'] = np.select(condiciones, opciones, default=None)


In [ ]:
mapa_ciudades = df_Ciudad.set_index('CODIGOCIUDAD')['NOMBRECIUDAD']
df_Estudiante['NOMBRECIUDAD'] = df_Estudiante['CODIGOCIUDAD'].map(mapa_ciudades)

columna_ciudad = df_Estudiante.pop('NOMBRECIUDAD')
posicion_insercion = df_Estudiante.columns.get_loc('CONTACTO_INCOMPLETO')
df_Estudiante.insert(loc=posicion_insercion, column='NOMBRECIUDAD', value=columna_ciudad)

conteo_estudiantes = df_Estudiante['CODIGOCIUDAD'].value_counts()
df_Ciudad['ESTUDIANTES'] = df_Ciudad['CODIGOCIUDAD'].map(conteo_estudiantes).fillna(0).astype(int)
#Ordenar por "Estudiantes"
df_Ciudad = df_Ciudad.sort_values(by='ESTUDIANTES', ascending=False)


In [ ]:
df_Estudiante.loc[df_Estudiante['ID'] == 6, 'CONTACTO_INCOMPLETO'] = 'CORREO'
df_Estudiante.to_csv(ruta_output_clean+'Estudiante.csv', index=False)
df_Ciudad.to_csv(ruta_output_clean+'Ciudad.csv', index=False)